# ## Prueba de API Key de OpenAI



In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Cargar variables de entorno del archivo .env
# IMPORTANTE: Asegúrate de tener un archivo llamado '.env' en la raíz de tu proyecto
# (al mismo nivel que la carpeta 'procesamiento' o donde ejecutes este notebook).
# El contenido del archivo .env debe ser:
# OPENAI_API_KEY='AQUÍ_VA_TU_API_KEY_DE_OPENAI'
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("Error: La variable de entorno OPENAI_API_KEY no fue encontrada.")
    print("Por favor, crea un archivo .env en la raíz de tu proyecto con tu API key.")
    print("Ejemplo de contenido para .env: OPENAI_API_KEY='sk-xxxxxxxxxx'")
else:
    try:
        client = OpenAI(api_key=api_key)
        print("Cliente OpenAI inicializado correctamente. ¡Tu API key parece funcionar!")
    except Exception as e:
        print(f"Error al inicializar el cliente OpenAI: {e}")
        print("Verifica que tu API key sea válida.")


Cliente OpenAI inicializado correctamente. ¡Tu API key parece funcionar!


In [3]:
if api_key: # Solo ejecutar si la API key se cargó
    pregunta_simple = "¿Cuál es la capital del Perú y un dato interesante sobre ella?"

    # System prompt de prueba (muy simplificado)
    # Para tu chatbot completo, usarías el system_prompt_v2 de chatbot_logic.py
    system_prompt_prueba = "Eres un asistente virtual que responde preguntas de forma concisa y amigable."

    mensajes_para_api = [
        {"role": "system", "content": system_prompt_prueba},
        {"role": "user", "content": pregunta_simple}
    ]

    try:
        print(f"\nEnviando pregunta a OpenAI: '{pregunta_simple}'")

        # Nota: El modelo "gpt-4.1-nano" que usas en chatbot_logic.py no parece ser un modelo estándar de OpenAI.
        # Para esta prueba, usaremos "gpt-3.5-turbo".
        # Si "gpt-4.1-nano" es un modelo específico al que tienes acceso (custom model o fine-tuned), puedes cambiarlo.
        completion = client.chat.completions.create(
            model="gpt-4.1-nano",  # Modelo estándar para prueba
            messages=mensajes_para_api,
            temperature=0.5,
            max_tokens=150
        )
        respuesta = completion.choices[0].message.content
        print(f"Respuesta de la API: {respuesta}")

    except Exception as e:
        print(f"Error al contactar con OpenAI: {e}")
        print("Verifica tu API key, conexión a internet, el modelo especificado y los parámetros de la solicitud.")
else:
    print("No se puede enviar la pregunta porque la API key no está configurada.")



Enviando pregunta a OpenAI: '¿Cuál es la capital del Perú y un dato interesante sobre ella?'
Respuesta de la API: La capital del Perú es Lima. Un dato interesante es que Lima es conocida como la "Ciudad de los Reyes" y fue fundada en 1535 por el conquistador Francisco Pizarro.


In [4]:
import os
import sys
from dotenv import load_dotenv
from openai import OpenAI

# Importar funciones y variables de chatbot_logic.py
# Esta importación asume que este notebook está en la misma carpeta que chatbot_logic.py
try:
    from chatbot_logic import (
        load_chunks_from_jsonl,
        send_question_to_openai,
        system_prompt_v2,
        extract_query_parameters, # Opcional, para pruebas separadas
        find_relevant_chunks      # Opcional, para pruebas separadas
    )
    print("Funciones de chatbot_logic.py importadas correctamente.")
except ImportError as e:
    print(f"Error al importar desde chatbot_logic.py: {e}")
    print("Asegúrate de que este notebook esté en la carpeta 'procesamiento/src_chatbot/' o ajusta el sys.path si es necesario.")
    # Si el notebook está en la raíz del proyecto, la importación sería:
    # from procesamiento.src_chatbot.chatbot_logic import (...)

# Cargar variables de entorno del archivo .env que debe estar en la raíz del proyecto
# (QLAB_CHATBOT_CORRUPCION/.env)
try:
    # Determinar la ruta raíz del proyecto desde la ubicación actual del notebook
    # Asumimos que el notebook está en QLAB_CHATBOT_CORRUPCION/procesamiento/src_chatbot/
    current_notebook_dir = os.path.dirname(os.path.abspath("__file__" if "__file__" in locals() else os.getcwd()))
    project_root_for_env = os.path.abspath(os.path.join(current_notebook_dir, '..', '..'))
    dotenv_path = os.path.join(project_root_for_env, '.env')

    if os.path.exists(dotenv_path):
        load_dotenv(dotenv_path=dotenv_path)
        print(f"Archivo .env cargado desde: {dotenv_path}")
    else:
        # Intento de carga alternativo (si el notebook se ejecuta desde la raíz)
        if load_dotenv():
            print("Archivo .env cargado desde una ubicación por defecto (posiblemente la raíz).")
        else:
            print(f"ADVERTENCIA: No se encontró el archivo .env en {dotenv_path} ni en la ubicación por defecto.")
            print("Por favor, crea un archivo .env en la raíz del proyecto (QLAB_CHATBOT_CORRUPCION/) con tu OPENAI_API_KEY.")
except Exception as e:
    print(f"Error cargando .env: {e}")


# Configuración del cliente OpenAI
api_key = os.getenv("OPENAI_API_KEY")
client = None

if not api_key:
    print("ERROR: OPENAI_API_KEY no encontrada en las variables de entorno.")
    print("Asegúrate de que el archivo .env esté configurado correctamente en la raíz del proyecto.")
else:
    try:
        client = OpenAI(api_key=api_key)
        print("Cliente OpenAI inicializado correctamente.")
    except Exception as e:
        print(f"Error al inicializar el cliente OpenAI: {e}")
        client = None # Asegurarse que client es None si falla

# Cargar los chunks de documentos
# La función load_chunks_from_jsonl en chatbot_logic.py calcula la ruta al archivo.
all_docs_chunks = []
if 'load_chunks_from_jsonl' in globals(): # Verificar si la importación fue exitosa
    try:
        all_docs_chunks = load_chunks_from_jsonl() # Esta función ya tiene la ruta al JSONL
        if not all_docs_chunks:
            print("ADVERTENCIA: No se cargaron chunks. Verifica la ruta del archivo 'salida_chunks_final.jsonl' definida en chatbot_logic.py y que el archivo exista y contenga datos.")
        elif len(all_docs_chunks) > 0:
            print(f"Se cargaron {len(all_docs_chunks)} chunks correctamente.")
    except Exception as e:
        print(f"Error al cargar chunks: {e}")
        all_docs_chunks = [] # Asegurarse que la lista está vacía si falla
else:
    print("ERROR: La función load_chunks_from_jsonl no pudo ser importada. No se pueden cargar los chunks.")



Funciones de chatbot_logic.py importadas correctamente.
Archivo .env cargado desde una ubicación por defecto (posiblemente la raíz).
Cliente OpenAI inicializado correctamente.
Cargados 10620 chunks desde c:\Users\alfie\OneDrive\Documents\GitHub\qlab_chatbot_corrupcion\output\salida_chunks_final.jsonl
Se cargaron 10620 chunks correctamente.


In [5]:
if client and all_docs_chunks:
    # --- PREGUNTA DE EJEMPLO ---
    # Puedes cambiar esta pregunta para probar diferentes escenarios:
    pregunta_usuario = "¿Qué informes de auditoría mencionan irregularidades en Cusco durante el año 2020?"
    # Otras preguntas para probar:
    # pregunta_usuario = "Resumen de la situación en Ancash para el 2018"
    # pregunta_usuario = "Háblame del informe 001-2016-2-0335"
    # pregunta_usuario = "Hay algo sobre Madre de Dios?" # Para probar 'needs_more_specificity'
    # pregunta_usuario = "Corrupción en Puno 2017"
    # pregunta_usuario = "Dime qué pasó en Lambayeque en 2019 con obras públicas"

    print(f"Pregunta del usuario: {pregunta_usuario}\n")

    # Historial de conversación (vacío para la primera pregunta o puedes simular uno)
    conversation_history = [
        # Ejemplo de historial:
        # {"role": "user", "content": "Hola"},
        # {"role": "assistant", "content": "Hola, soy el Chatbot Corrupción. ¿En qué puedo ayudarte?"}
    ]

    print("Enviando pregunta al chatbot (esto puede tardar unos segundos)...")
    try:
        # Llamar a la función principal del chatbot
        respuesta_chatbot = send_question_to_openai(
            question=pregunta_usuario,
            all_docs_chunks=all_docs_chunks,
            conversation_history=conversation_history,
            openai_client=client, # El cliente OpenAI inicializado
            system_prompt=system_prompt_v2  # El system_prompt_v2 importado de chatbot_logic.py
        )

        print("\n========= RESPUESTA DEL CHATBOT =========")
        print(respuesta_chatbot)
        print("==========================================")

        # Opcional: Probar la extracción de parámetros y búsqueda de chunks por separado para depuración
        print("\n--- Detalles del procesamiento interno (depuración) ---")
        params_debug = extract_query_parameters(pregunta_usuario)
        print(f"Parámetros extraídos por extract_query_parameters: {params_debug}")

        if params_debug.get("is_specific_enough", False):
            retrieval_result_debug = find_relevant_chunks(pregunta_usuario, all_docs_chunks, max_chunks=5) # max_chunks bajo para debug
            if retrieval_result_debug.get("needs_more_specificity"):
                print("find_relevant_chunks indica: Necesita más especificidad.")
            elif not retrieval_result_debug.get("chunks"):
                print(f"find_relevant_chunks: No se encontraron chunks. Razón: {retrieval_result_debug.get('no_data_for_filter') or retrieval_result_debug.get('no_data_for_filter_after_score', 'desconocida')}")
                if retrieval_result_debug.get("params"): print(f"  Parámetros usados por find_relevant_chunks: {retrieval_result_debug.get('params')}")
            else:
                print(f"find_relevant_chunks: Encontrados {len(retrieval_result_debug.get('chunks', []))} chunks relevantes (máx. 5 para debug).")
                # Puedes descomentar lo siguiente para ver detalles de los chunks recuperados:
                # for i, chunk_info in enumerate(retrieval_result_debug.get('chunks', [])):
                #     print(f"  Chunk {i+1}: Informe {chunk_info['metadata'].get('numero_informe', 'N/A')}, Tipo: {chunk_info.get('source_field', 'N/A')}, Score (si aplica): {chunk_info.get('score_debug', 'N/A')}")
        else:
            print("La pregunta no fue lo suficientemente específica según extract_query_parameters.")


    except Exception as e:
        print(f"Ocurrió un error al procesar la pregunta con send_question_to_openai: {e}")
        print("Verifica tu API key, el modelo 'gpt-4.1-nano' (si es un modelo al que tienes acceso o si necesitas cambiarlo a uno estándar como 'gpt-3.5-turbo'), y tu conexión a internet.")

elif not client:
    print("El cliente de OpenAI no está inicializado. Verifica tu API key en el archivo .env.")
elif not all_docs_chunks:
    print("No hay chunks cargados. No se puede procesar la pregunta. Verifica que 'salida_chunks_final.jsonl' exista y sea accesible.")
else:
    print("Error desconocido en la configuración inicial.")



Pregunta del usuario: ¿Qué informes de auditoría mencionan irregularidades en Cusco durante el año 2020?

Enviando pregunta al chatbot (esto puede tardar unos segundos)...

========= RESPUESTA DEL CHATBOT =========
Según los informes de auditoría proporcionados, en el año 2020 en la región de Cusco se identificaron algunas irregularidades y observaciones relevantes:

1. **Informe 017-2020-2-1627 (Municipalidad Distrital de San Jerónimo):** Este informe tiene como objetivo determinar si la ejecución del contrato para la creación de servicios de salud en la margen derecha del río Huatanay en San Jerónimo, Cusco, se realizó en cumplimiento de la normativa aplicable. Sin embargo, en el fragmento proporcionado no se detallan observaciones específicas sobre irregularidades, solo el objetivo de la auditoría.

2. **Informe 048-2020-2-0385 (Municipalidad Provincial del Cusco):** Este informe evaluó el proyecto de inversión para el mejoramiento de la transitabilidad vehicular y peatonal en la ca